In [1]:
import pandas as pd
import os

root = './'
folder_path = root+'Input files/' +'Historical'

In [1]:
dataframes = []

# Read all files in the folder and concatenate them
for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
        dataframes.append(df)

# Concatenate all dataframes
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.head(200)

NameError: name 'os' is not defined

In [3]:
combined_df_copy = combined_df.copy()

combined_df_copy['Date'] = pd.to_datetime(combined_df_copy['Date'], errors='coerce')

combined_df_copy.head()

NameError: name 'combined_df' is not defined

In [19]:
# Define the date range
start_date = pd.to_datetime('2018-05-01')
end_date = pd.to_datetime('2020-04-01')

# Initialize sets to store cost centres with issues
cost_centres_invalid_start = set()
cost_centres_invalid_end = set()

# Check each cost centre's date range in the copied dataframe
for cost_centre in combined_df_copy['Cost_Centre'].unique():
    cost_centre_dates = combined_df_copy[combined_df_copy['Cost_Centre'] == cost_centre]['Date']

    if cost_centre_dates.min() > start_date:
        cost_centres_invalid_start.add(cost_centre)

    if cost_centre_dates.max() < end_date:
        cost_centres_invalid_end.add(cost_centre)

# Report the cost centres with start dates after the specified start date
print(f"Cost centres with start dates after {start_date.date()}:")
for cost_centre in cost_centres_invalid_start:
    print(cost_centre)

# Report the cost centres with end dates before the specified end date
print(f"\nCost centres with end dates before {end_date.date()}:")
for cost_centre in cost_centres_invalid_end:
    print(cost_centre)

Cost centres with start dates after 2018-05-01:
INF_CM_Scotland
INF_PM_Scotland

Cost centres with end dates before 2020-04-01:


In [3]:
# Define the columns to check for missing values
columns_to_check = ['direct_costs', 'overheads', 'total_headcount', 'income_ex']

# Initialize a set to store cost centres with missing values
cost_centres_with_missing_values = set()

# Check for missing values in the specified columns
for cost_centre in combined_df['Cost_Centre'].unique():
    cost_centre_data = combined_df[combined_df['Cost_Centre'] == cost_centre]
    if cost_centre_data[columns_to_check].isnull().any().any():
        cost_centres_with_missing_values.add(cost_centre)

# Report the cost centres with missing values in the specified columns
print("\nCost centres with missing values in 'direct_costs', 'overheads', 'total_headcount', 'income_ex':")
for cost_centre in cost_centres_with_missing_values:
    print(cost_centre)


Cost centres with missing values in 'direct_costs', 'overheads', 'total_headcount', 'income_ex':


In [4]:
combined_df['income_ex_fcst12'] = pd.NA
combined_df['direct_costs_fcst12'] = pd.NA
combined_df['overheads_fcst12'] = pd.NA
combined_df['total_headcount_fcst12'] = pd.NA

In [5]:
combined_df = combined_df.drop(columns = ['Unnamed: 0'])

In [6]:
main_data_path = 'combined_file_fcst2020added.csv'
df_main = pd.read_csv(main_data_path)

In [7]:
df_main['Date'] = pd.to_datetime(df_main['Date'], format='%d/%m/%Y')

In [8]:
adamcl_data_path = 'combined_file_amclad_fcst2023.csv'
df_adamcl = pd.read_csv(adamcl_data_path)
df_adamcl['Date'] = pd.to_datetime(df_adamcl['Date'], format='%Y-%m-%d')

In [9]:
dfs = pd.concat([df_main, combined_df, df_adamcl], ignore_index=True)
dfs = dfs.sort_values(by=['Cost_Centre', 'Date'])

In [10]:
summary = dfs.groupby('Cost_Centre').agg(
    start_date=('Date', 'min'),
    end_date=('Date', 'max'),
    missing_income_ex=('income_ex', lambda x: x.isna().any()),
    missing_direct_costs=('direct_costs', lambda x: x.isna().any()),
    missing_overheads=('overheads', lambda x: x.isna().any()),
    missing_total_headcount=('total_headcount', lambda x: x.isna().any())
).reset_index()
summary.to_csv('cost_centre_summary.csv', index=False)

In [20]:
dfs.to_csv('dfs.csv', index=True)

In [11]:
start_date = pd.to_datetime('2018-05-01')
end_date = pd.to_datetime('2024-04-01')

# Initialize sets to store cost centres with issues
cost_centres_invalid_start = set()
cost_centres_invalid_end = set()

# Check each cost centre's date range in the copied dataframe
for cost_centre in dfs['Cost_Centre'].unique():
    cost_centre_dates = dfs[dfs['Cost_Centre'] == cost_centre]['Date']

    if cost_centre_dates.min() > start_date:
        cost_centres_invalid_start.add(cost_centre)

    if cost_centre_dates.max() < end_date:
        cost_centres_invalid_end.add(cost_centre)

# Report the cost centres with start dates after the specified start date
print(f"Cost centres with start dates after {start_date.date()}:")
for cost_centre in cost_centres_invalid_start:
    print(cost_centre)

# Report the cost centres with end dates before the specified end date
print(f"\nCost centres with end dates before {end_date.date()}:")
for cost_centre in cost_centres_invalid_end:
    print(cost_centre)

Cost centres with start dates after 2018-05-01:
CM_Co
NR_PF
INF_CM_Midlands
INF_PMO_Scotland
ADV_AdvCo
CM_NorthEast
ADV_Disp
NR_NR_North
INF_PMO_Midlands
INF_PM_Midlands
INF_Co
PM_North_East
CM_Cambridge
INF_CM_Scotland
INF_PM_Scotland
PM_Project_Controls
ADV_Proc

Cost centres with end dates before 2024-04-01:


In [2]:
#cost_centres_to_exclude = ['...']  # Update with actual cost centres
cost_centres_to_exclude = list(cost_centres_invalid_start)
# Exclude the specified cost centres
filtered_df = dfs[~dfs['Cost_Centre'].isin(cost_centres_to_exclude)]

filtered_df = filtered_df.reset_index(drop= True)
filtered_df.head(200)

NameError: name 'cost_centres_invalid_start' is not defined

In [28]:
filtered_df.to_csv('hist_df.csv', index=False)

In [29]:
summary = filtered_df.groupby('Cost_Centre').agg(
    start_date=('Date', 'min'),
    end_date=('Date', 'max'),
    missing_income_ex=('income_ex', lambda x: x.isna().any()),
    missing_direct_costs=('direct_costs', lambda x: x.isna().any()),
    missing_overheads=('overheads', lambda x: x.isna().any()),
    missing_total_headcount=('total_headcount', lambda x: x.isna().any())
).reset_index()
summary.to_csv('cost_centre_summary2.csv', index=False)

In [34]:
combined_folder = root+'Combined files/'
combined_csv_path = combined_folder + 'hist_df_all.csv'
filtered_df.to_csv(combined_csv_path, index=False)

In [12]:
combined_folder = root+'Combined files/'
combined_csv_path = combined_folder + 'hist_df_all.csv'
dfs.to_csv(combined_csv_path, index=False)